# Building MOP-Terphenyl Polymers using mbuild

Here we use `mbuild` to read in a SMILES string of a terphenyl monomer and build an n-residue polymer from the monomer.

In [1]:
import mbuild as mb
import numpy as np
import subprocess
from mbuild.lib.recipes.polymer import Polymer
import warnings
warnings.filterwarnings('ignore')

First we get the MOP-terphenyl monomer from a smiles string.

In [2]:
comp = mb.load('C(=O)c1ccc(c2c(cc(OC)cc2)(c3cccc(c3)[C@@H](N)C))cc1', smiles = True, name="MOP")

I output all the indexes of hydrogen atoms because we will uses these indexes for extending the polymer later on.

In [3]:
for i, atom in enumerate(comp):
    if atom.name == "H":
        print(i, atom)

25 <H pos=([-0.2453 -0.5566  0.0782]), 1 bonds, id: 139773138644896>
26 <H pos=([-0.1188 -0.3599  0.1631]), 1 bonds, id: 139773138644944>
27 <H pos=([-0.0695 -0.1196  0.1639]), 1 bonds, id: 139773138674112>
28 <H pos=([ 0.0728  0.3003 -0.0153]), 1 bonds, id: 139773138674352>
29 <H pos=([-0.1463  0.6895  0.0515]), 1 bonds, id: 139773138674592>
30 <H pos=([-0.2465  0.5765  0.1501]), 1 bonds, id: 139773138674832>
31 <H pos=([-0.2739  0.5909 -0.0294]), 1 bonds, id: 139773138675072>
32 <H pos=([-0.3456  0.3696  0.0561]), 1 bonds, id: 139773138675312>
33 <H pos=([-0.3889  0.13    0.0462]), 1 bonds, id: 139773138675552>
34 <H pos=([-0.0317 -0.0545 -0.2148]), 1 bonds, id: 139773138675792>
35 <H pos=([ 0.1565 -0.2054 -0.2646]), 1 bonds, id: 139773138676032>
36 <H pos=([ 0.3579 -0.205  -0.1237]), 1 bonds, id: 139773138676272>
37 <H pos=([0.1828 0.1017 0.1242]), 1 bonds, id: 139773138676512>
38 <H pos=([0.3972 0.0289 0.1781]), 1 bonds, id: 139773138676752>
39 <H pos=([ 0.4395 -0.2537  0.0963]), 1

`mbuild` comes with a nice tool to visualize Compounds built into jupyter-notebooks. Using the object from `py3Dmol` we can coloro the atoms to identify the indices needed to make substiutions when building the polymer.

In [4]:
view = comp.visualize(show_ports=True)
style = {
                "stick": {"radius": 0.2, "color": "grey"},
                "sphere": {"scale": 0.3, "color" : "black"},
    }
view.setStyle({'model': -1, 'serial':26},style)
view.setStyle({'model': -1, 'serial':41},style)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

We also make the two capping compounds using SMILES strings:

In [5]:
cap_o = mb.load('CO', smiles = True)
cap_o.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
cap_n = mb.load('CC(C)(C)OC=O', smiles = True)
cap_n.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Here we use the `Polymer` object to build a hexamer from the molecules we built in the previous cells. `Polymer.add_monomer()` is used to add the monomers to the polymer object. `Polymer.add_end_groups()` adds the capping groups to the object with `"head"` and `"tail"` labels for the each end of the polymer. `replace = True` will replace the specified atoms with the next residue. `indices` is used to specify which atom will be replaced in each group. Finally, when we call `Polymer.build()`, the polymer is built with the specified `n` residues and the provided capping residues. `sequence` is used if multiple monomers are provided.

In [7]:
chain = Polymer()
chain.add_monomer(compound=comp,
                  indices=[25, 40],
                  separation=.15,
                  replace=True,
                  # orientation = [[0,-1,0],[1,0,0]]
                 )
chain.add_end_groups(compound = cap_o,
                     index = -1,
                     separation=0.15,
                     label="head",
                     duplicate = False
                    )

chain.add_end_groups(compound = cap_n,
                     index = -1,
                     separation=0.15,
                     label="tail",
                     duplicate = False
                    )

chain.build(n=6, sequence='A')

In [8]:
chain.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [9]:
for p in chain.particles():
    print(p)

<C pos=([-0.3002 -0.4911  0.0117]), 3 bonds, id: 139773002123680>
<O pos=([-0.3879 -0.5407 -0.0639]), 1 bonds, id: 139773002152880>
<C pos=([-0.2717 -0.3465  0.0106]), 3 bonds, id: 139773002152976>
<C pos=([-0.1735 -0.2945  0.0961]), 3 bonds, id: 139773002153120>
<C pos=([-0.145  -0.1577  0.0963]), 3 bonds, id: 139773002153264>
<C pos=([-0.2138 -0.0707  0.0101]), 3 bonds, id: 139773002153408>
<C pos=([-0.1831  0.0744  0.0093]), 3 bonds, id: 139773002153552>
<C pos=([-0.0513  0.1234 -0.0091]), 3 bonds, id: 139773002121472>
<C pos=([-0.0275  0.2617 -0.0011]), 3 bonds, id: 139773002153600>
<C pos=([-0.1319  0.3517  0.0222]), 3 bonds, id: 139773002153792>
<O pos=([-0.1017  0.4885  0.0285]), 2 bonds, id: 139773002153936>
<C pos=([-0.1977  0.5912  0.0514]), 4 bonds, id: 139773002174624>
<C pos=([-0.2626  0.3026  0.038 ]), 3 bonds, id: 139773002174768>
<C pos=([-0.2876  0.1655  0.0315]), 3 bonds, id: 139773002174912>
<C pos=([ 0.0623  0.0335 -0.0408]), 3 bonds, id: 139773002175056>
<C pos=([ 

In [10]:
# no need for rotation of peptide bonds
#for bond in chain.bonds():
#    if bond[0].name == "N":
#        if bond[1].n_direct_bonds == 3:
#            print("Rotating bond:", bond, "by", np.pi/2)
#            chain.rotate_dihedral(bond, np.pi/2)
#            
#    if bond[1].name == "N":
#        if bond[0].n_direct_bonds == 3:
#            print("Rotating bond:", bond, "by", np.pi/2)
#            chain.rotate_dihedral(bond, np.pi/2)       

In [11]:
chain.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Next we add specific residue labels for the componenets of the `Polymer` object. Here we label the monomers as HEX and the capping moieties as CAP.

In [12]:
print(chain.labels)
for label in chain.labels["monomer"]:
    label.name = "HEX"
    print(label)
for label in chain.labels["Compound"]:
    label.name = "CAP"
    print(label)

OrderedDict([('monomer', [<Compound 44 particles, 46 bonds, non-periodic, id: 139773002047440>, <Compound 44 particles, 46 bonds, non-periodic, id: 139773002063936>, <Compound 44 particles, 46 bonds, non-periodic, id: 139773001933392>, <Compound 44 particles, 46 bonds, non-periodic, id: 139773001521808>, <Compound 44 particles, 46 bonds, non-periodic, id: 139773001686752>, <Compound 44 particles, 46 bonds, non-periodic, id: 139773001339760>]), ('monomer[0]', <Compound 44 particles, 46 bonds, non-periodic, id: 139773002047440>), ('monomer[1]', <Compound 44 particles, 46 bonds, non-periodic, id: 139773002063936>), ('monomer[2]', <Compound 44 particles, 46 bonds, non-periodic, id: 139773001933392>), ('monomer[3]', <Compound 44 particles, 46 bonds, non-periodic, id: 139773001521808>), ('monomer[4]', <Compound 44 particles, 46 bonds, non-periodic, id: 139773001686752>), ('monomer[5]', <Compound 44 particles, 46 bonds, non-periodic, id: 139773001339760>), ('Compound', [<Compound 5 particles,

We save these as a pdb file and provide the names of the residues to include in the file.

In [13]:
chain.save("mop_hexamer_mbuild.pdb", overwrite=True, residues=["HEX", "CAP"])

Using Gromacs I generate a new gro file with the correct residue labels. `mbuild` doesn't seem to label residues correctly when writing `.gro` files.

In [14]:
! gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_hexamer_mbuild.pdb -o mop_hexamer_mbuild.gro

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 285 atoms
Volume: 37.1134 nm^3, corresponds to roughly 16700 electrons
No velocities found

Back Off! I just backed up mop_hexamer_mbuild.gro to ./#mop_hexamer_mbuild.gro.3#

GROMACS reminds you: "People are DNA's way of making more DNA." (Edward O. Wilson)



Lastly, I use openBabel to generate a `.mol` file for use in the OpenFF parameter assignment workflow.

In [15]:
! obabel -ipdb mop_hexamer_mbuild.pdb -omol mop_hexamer_mbuild.mol -O mop_hexamer_mbuild.mol

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


# Building arbitrary n-mers of MOP monomer

Now we put it all together to generate a tetramer, hexamer and octamer PMP terphenyl foldamer:

In [16]:
n_residues = [2,3,4,6,8,10]
residue_labels = ["DIM", "TRI","TET", "HEX", "OCT", "DEC"]
file_names = ["mop_dimer_mbuild", "mop_trimer_mbuild", "mop_tetramer_mbuild", "mop_hexamer_mbuild", "mop_octamer_mbuild", "mop_decamer_mbuild"]
for i in range(len(n_residues)):
    # Generate polymer
    chain = Polymer()
    chain.add_monomer(compound=comp,
                      indices=[25, 40],
                      separation=.15,
                      replace=True,
                      # orientation = [[0,-1,0],[1,0,0]]
                     )
    chain.add_end_groups(compound = cap_o,
                         index = -1,
                         separation=0.15,
                         label="head",
                         duplicate = False
                        )

    chain.add_end_groups(compound = cap_n,
                         index = -1,
                         separation=0.15,
                         label="tail",
                         duplicate = False
                        )

    chain.build(n=n_residues[i], sequence='A')

    # Rotate peptide bond
    #for bond in chain.bonds():
    #    if bond[0].name == "N":
    #        if bond[1].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)

    #    if bond[1].name == "N":
    #        if bond[0].n_direct_bonds == 3:
    #            print("Rotating bond:", bond, "by", np.pi/2)
    #            chain.rotate_dihedral(bond, np.pi/2)  


    # Relabel chains
    for label in chain.labels["monomer"]:
        label.name = residue_labels[i]
        print(label)
    for label in chain.labels["Compound"]:
        label.name = "CAP"
        print(label)
            
            
    chain.save(file_names[i] + ".pdb", overwrite=True, residues=[residue_labels[i], "CAP"])
    
    subprocess.run(["gmx", "editconf", "-f", file_names[i] + ".pdb", "-o", file_names[i]+ ".gro"])
    subprocess.run(["obabel", "-ipdb", file_names[i] + ".pdb", "-omol", file_names[i] + ".mol",  "-O", file_names[i] + ".mol"])
    subprocess.run(["renumber_pdb_atoms", "-f", file_names[i] + ".pdb", "-o", file_names[i] + "_renum.pdb"])

<DIM 44 particles, 46 bonds, non-periodic, id: 139772972745728>
<DIM 44 particles, 46 bonds, non-periodic, id: 139772972748224>
<CAP 5 particles, 4 bonds, non-periodic, id: 139772973128384>
<CAP 16 particles, 15 bonds, non-periodic, id: 139772972657200>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 109 atoms
Volume: 8.36704 nm^3, corresponds to roughly 3700 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_dimer_mbuild.pdb -o mop_dimer_mbuild.gro


Back Off! I just backed up mop_dimer_mbuild.gro to ./#mop_dimer_mbuild.gro.1#

GROMACS reminds you: "What's Your Definition Of Dirty ?" (G. Michael)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<TRI 44 particles, 46 bonds, non-periodic, id: 139772973327936>
<TRI 44 particles, 46 bonds, non-periodic, id: 139772973330144>
<TRI 44 particles, 46 bonds, non-periodic, id: 139772973444736>
<CAP 5 particles, 4 bonds, non-periodic, id: 139772972201920>
<CAP 16 particles, 15 bonds, non-periodic, id: 139772972163616>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 153 atoms
Volume: 11.9203 nm^3, corresponds to roughly 5300 electrons
No velocities found


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_trimer_mbuild.pdb -o mop_trimer_mbuild.gro


Back Off! I just backed up mop_trimer_mbuild.gro to ./#mop_trimer_mbuild.gro.1#

GROMACS reminds you: "I always think there is something foreign about jolly phrases at breakfast." (Mr. Carson in Downtown Abbey)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<TET 44 particles, 46 bonds, non-periodic, id: 139772973753152>
<TET 44 particles, 46 bonds, non-periodic, id: 139772973755648>
<TET 44 particles, 46 bonds, non-periodic, id: 139772973377184>
<TET 44 particles, 46 bonds, non-periodic, id: 139772971985840>
<CAP 5 particles, 4 bonds, non-periodic, id: 139772973680480>
<CAP 16 particles, 15 bonds, non-periodic, id: 139772973680192>
Note that major changes are planned in future for editconf, to improve usability and utility.
Read 197 atoms
Volume: 19.6508 nm^3, corresponds to roughly 8800 electrons
No velocities found


...

                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_octamer_mbuild.pdb -o mop_octamer_mbuild.gro


Back Off! I just backed up mop_octamer_mbuild.gro to ./#mop_octamer_mbuild.gro.2#

GROMACS reminds you: "Count the Bubbles In Your Hair" (The Breeders)

*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted


<DEC 44 particles, 46 bonds, non-periodic, id: 139775712905056>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775712905008>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775695923664>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775695560224>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775695200944>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775695394560>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775695006544>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775694634720>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775694787440>
<DEC 44 particles, 46 bonds, non-periodic, id: 139775694411712>
<CAP 5 particles, 4 bonds, non-periodic, id: 139775712771568>
<CAP 16 particles, 15 bonds, non-periodic, id: 139775712820576>


                     :-) GROMACS - gmx editconf, 2022.4 (-:

Executable:   /usr/local/bin/gmx
Data prefix:  /usr/local
Working dir:  /home/tfobe/Research/heteropolymer_simulations/examples/build_polymer/mop
Command line:
  gmx editconf -f mop_decamer_mbuild.pdb -o mop_decamer_mbuild.gro


Back Off! I just backed up mop_decamer_mbuild.gro to ./#mop_decamer_mbuild.gro.2#

GROMACS reminds you: "Two chemists walk into a bar. The first one says, 'I'll have some H2O.'. The second one says, 'I'll have some H2O, too'. He dies." (Anonymous)



Note that major changes are planned in future for editconf, to improve usability and utility.
Read 461 atoms
Volume: 121.016 nm^3, corresponds to roughly 54400 electrons
No velocities found


*** Open Babel Error  in ReadMolecule
  ERROR: not a valid PDB file

1 molecule converted
